In [1]:
import sys
import os
from src.model import model
import numpy as np
from loguru import logger
import torch

Load the data that is generated from the open-loop optimization

In [2]:
# load the data
path = '../data_result/raw_data/VDP_beta_0.1_grid_combined.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-09-19 16:06:32.730 | INFO     | __main__:<module>:4 - Loaded data with shape: (1800,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


## SSN(line-search) method for outer weights ##


In [3]:
training_percentage = 0.9

optimizer = 'SSN'
activation = torch.relu
power = 2.1
regularization = (5, 0.01)
lr = 0.01

loss_weights = (1.0, 1.0)
th = 0.5
verbose = True
train_outerweights = True

# n_neurons = 100
# weights = np.random.randn(n_neurons, 2) * 0.1
# bias = np.random.randn(n_neurons) * 0.1
# outer_weights = np.random.randn(n_neurons) * 0.1
# regularization = (5, 0.01)

In [4]:
# Initializing the model
vdp_model = model(
    training_percentage, 
    optimizer, activation, power, regularization, lr, 
    loss_weights, th,
    train_outerweights = True
)

data_train, data_valid = vdp_model._prepare_data(data)

print("train_outerweights:", vdp_model.train_outerweights)
print("optimizer_type:", vdp_model.optimizer_type)


2025-09-12 21:11:18 | INFO     | src.model:_configure_logger:104 - VDPModel initialized
2025-09-12 21:11:18 | INFO     | src.model:_prepare_data:149 - Training set: 1620 samples, Validation set: 180 samples
2025-09-12 21:11:18 | INFO     | src.model:_prepare_data:152 - Data ranges - x: [-3.00, 3.00], v: [0.00, 10.96], dv: [-13.19, 13.19]
train_outerweights: True
optimizer_type: SSN


In [5]:
# train the model
artifacts = vdp_model.train(
    data_train, data_valid,
    iterations=10000,
    display_every=1000
)

2025-09-12 21:11:18 | INFO     | src.model:train:313 - Starting network training session
2025-09-12 21:11:19 | INFO     | src.model:_setup_optimizer:226 - Using SSN optimizer with alpha=0.01, gamma=5, th=0.5
2025-09-12 21:11:19 | INFO     | src.model:train:330 - Training model, saving to /Users/ruizhechao/Documents/NNforHJB/train_history
2025-09-12 21:11:19 | INFO     | src.model:train:331 - Training hyperparameters: iterations=10000, batch_size=1620, display_every=1000
2025-09-12 21:11:19 | INFO     | src.model:train:332 - Loss weights: value=1.0, gradient=1.0
2025-09-12 21:11:19 | WARNING  | src.ssn:step:283 - Theta reached 4.8e-16. Breaking line search loop.
2025-09-12 21:11:19 | INFO     | src.model:train:380 - Epoch 0: Train Loss = 37.321156, Val Loss = 603.994897
2025-09-12 21:11:19 | WARNING  | src.ssn:step:283 - Theta reached 4.8e-16. Breaking line search loop.
2025-09-12 21:11:19 | WARNING  | src.ssn:step:283 - Theta reached 4.8e-16. Breaking line search loop.
2025-09-12 21:11

RuntimeError: SSN optimizer failed repeatedly - stopping to prevent infinite loop

## SSN(line-search) for the outer weights ##

In [ ]:
vdp_model = model_outerweights(data, torch.relu, 2.0, regularization, 'SSN', loss_weights=(1.0, 0.0))
model_result, weight, bias, output_weight = vdp_model.train(
    inner_weights=weights, inner_bias=bias, outer_weights=outer_weights,
    iterations=10000,
    display_every=500
)

NameError: name 'model_outerweights' is not defined

## SSN(trust-region) for the outer weights

In [ ]:
vdp_model = model_outerweights(data, torch.relu, 2.0, regularization, 'SSN_TR', loss_weights=(1.0, 0.0))
model_result, weight, bias, output_weight = vdp_model.train(
    inner_weights=weights, inner_bias=bias, outer_weights=outer_weights,
    iterations=10000,
    display_every=500
)